In [1]:
!git clone https://github.com/openai/shap-e

Cloning into 'shap-e'...


In [2]:
%cd shap-e

c:\Users\gerpv\Desktop\predpp\preddippract\shap-e


C:\Users\gerpv\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!pip install -e .
!pip install googletrans==4.0.0-rc1
!pip install matplotlib
!pip install scikit-learn

Obtaining file:///C:/Users/gerpv/Desktop/predpp/preddippract/shap-e
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning https://github.com/openai/CLIP.git to c:\users\gerpv\appdata\local\temp\pip-install-b9zu0406\clip_655169e6aba446879e03b415ef1aff78
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: shap-e
    Found existing installation: shap-e 0.0.0
    Uninstalling shap-e-0.0.0:
      Successfully uninstalled shap-e-0.0.0
  Running setup.py develop for shap-e


DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

In [4]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from googletrans import Translator
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))
# Создаем объект переводчика
translator = Translator()

# Функция для перевода текста с русского на английский
def translate_to_english(text):
    translation = translator.translate(text, src='ru', dest='en')
    return translation.text

c:\Users\gerpv\Desktop\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
c:\Users\gerpv\Desktop\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
c:\Users\gerpv\Desktop\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
c:\Users\gerpv\Desktop\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

c:\Users\gerpv\Desktop\predpp\preddippract\shap-e\shap_e\models\download.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path, map_location=device)
1

  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

c:\Users\gerpv\Desktop\predpp\preddippract\shap-e\shap_e\models\download.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path, map_location=device)


In [ ]:
batch_size = 1
guidance_scale = 15.0
prompt = "a red plane"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
render_mode = 'nerf' 
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

In [ ]:
from google.colab import files
import torch
import ipywidgets as widgets
from IPython.display import display, FileLink
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from googletrans import Translator
import imageio
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# Инициализация переводчика
translator = Translator()

# Функция для перевода текста с русского на английский
def translate_to_english(text):
    translation = translator.translate(text, src='ru', dest='en')
    return translation.text

# Создание виджетов для ввода параметров
prompt_widget = widgets.Text(value='красный самолет', description='Prompt:', disabled=False)
guidance_scale_widget = widgets.FloatSlider(value=15.0, min=1.0, max=30.0, step=0.1, description='Guidance Scale:', disabled=False)
render_mode_widget = widgets.Dropdown(options=['nerf', 'stf'], value='nerf', description='Render Mode:', disabled=False)
size_widget = widgets.IntSlider(value=64, min=32, max=256, step=1, description='Size:', disabled=False)

# Создание кнопки для начала генерации
generate_button = widgets.Button(description="Generate")

# Создание кнопки для сохранения GIF
save_button = widgets.Button(description="Save GIF")

# Инициализация переменных для хранения сгенерированных изображений
generated_images = []

# Определение функции, которая будет вызываться при нажатии на кнопку генерации
def on_generate_button_clicked(b):
    global generated_images
    prompt = prompt_widget.value
    translated_prompt = translate_to_english(prompt)
    guidance_scale = guidance_scale_widget.value
    render_mode = render_mode_widget.value
    size = size_widget.value

    latents = sample_latents(
        batch_size=batch_size,
        model=model,
        diffusion=diffusion,
        guidance_scale=guidance_scale,
        model_kwargs=dict(texts=[translated_prompt] * batch_size),
        progress=True,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=64,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    )

    cameras = create_pan_cameras(size, device)
    generated_images = []
    for i, latent in enumerate(latents):
        images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
        generated_images.append(images)
        display(gif_widget(images))

    # Визуализация латентных представлений
    visualize_latents(latents)

# Определение функции, которая будет вызываться при нажатии на кнопку сохранения
def on_save_button_clicked(b):
    if generated_images:
        output_path = 'generated_gif.gif'
        imageio.mimsave(output_path, generated_images[0], fps=10)
        print(f"GIF сохранен в {output_path}")
        # Создание ссылки для скачивания сохраненного GIF
        files.download(output_path)
    else:
        print("Изображения еще не сгенерированы. Пожалуйста, сначала сгенерируйте изображения.")

# Функция для визуализации латентных представлений
def visualize_latents(latents):
    # Преобразование латентных представлений в двумерный массив с помощью PCA
    pca = PCA(n_components=2)
    latents_2d = pca.fit_transform(latents.cpu().numpy())

    plt.figure(figsize=(10, 5))
    plt.scatter(latents_2d[:, 0], latents_2d[:, 1], c='blue', marker='o')
    plt.title('PCA of Latent Representations')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.grid(True)
    plt.show()

# Привязка функций к кнопкам
generate_button.on_click(on_generate_button_clicked)
save_button.on_click(on_save_button_clicked)

# Создание разделов для организации интерфейса
input_section = widgets.VBox([prompt_widget, guidance_scale_widget, render_mode_widget, size_widget])
button_section = widgets.HBox([generate_button, save_button])

# Отображение виджетов и кнопок
display(input_section, button_section)